In [ ]:
# NECESITA KAGGLE, y seleccionar una GPU H100 PARA CORRER ESTO
# Settings -> Accelerator -> GPU H100

import keras_nlp
import tensorflow as tf
import keras_hub
import keras

# Ignore el mensaje "WARNING: Unable to register cuBLAS factory"

2025-08-07 22:06:12.250707: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1754604372.272316     125 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1754604372.279016     125 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [ ]:
# describir la GPU que acaba de conseguir
!nvidia-smi

Thu Aug  7 22:06:15 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 560.35.03              Driver Version: 560.35.03      CUDA Version: 12.6     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla P100-PCIE-16GB           Off |   00000000:00:04.0 Off |                    0 |
| N/A   36C    P0             26W /  250W |       0MiB /  16384MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [ ]:
# Configurar el dtype por defecto a  bfloat16 para mejorar performance y bajar uso de memoria
keras.config.set_dtype_policy("bfloat16")

In [ ]:
import os
# ACA ponga su usuario y clave de kaggle
os.environ["KAGGLE_USERNAME"] = "mi_usuario_de_kaggle"
os.environ["KAGGLE_KEY"] = "mi_clave_de_kaggle"

In [ ]:
# Carganvidia-smir Gemma de 2 mil millones (2 Billion) de parametros.
# Es el modelo más grande que entra en la version gratis de Kaggle con GPU.
# Use keras_hub.models.GemmaCausalLM.presets.keys() si quiere ver todos los presets.
gemma_lm = keras_hub.models.GemmaCausalLM.from_preset("gemma_instruct_2b_en")

I0000 00:00:1754604378.454754     125 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 15513 MB memory:  -> device: 0, name: Tesla P100-PCIE-16GB, pci bus id: 0000:00:04.0, compute capability: 6.0
normalizer.cc(51) LOG(INFO) precompiled_charsmap is empty. use identity normalization.


In [ ]:
# Esto es un hack para generar una salida mas linda. El modelo va a para luego
# de generar este token.
# Aqui usamos ")" porque en las instrucciones le estamos pidiendo que genere ")" luego de la categoria.
# CUIDADO: Si cambia el token, y el toke id = 3 eso significa "desconocido", y significa
# que no va a servir como stop token (no va a parar).
stop_token = (gemma_lm.preprocessor.tokenizer.token_to_id(")"),)
stop_token

(235275,)

In [ ]:
# Este es un ejemplo (pobre) de few-shot learning.
# Muy probablemente necesite agregar mas ejemplos.
prompt_text = """
<start_of_turn>user
This set of news with a category. Each news starts with <EXAMPLE> and ends with </EXAMPLE>, each news belongs to a single category that stars with <CATEGORY> and ends with </CATEGORY>:

<EXAMPLE>
<INPUT>
Se trabó el acuerdo entre Karina Milei y los Macri para compartir las listas en la Capital.
La negociación entre LLA y Pro se interrumpió porque no logran consensuar el reparto de lugares en la nómina de diputados nacionales; malestar y reproches cruzados.
La negociación entre Karina Milei y los Macri para que La Libertad Avanza (LLA) y Pro conformen un frente electoral en la Capital para competir en las legislativas nacionales del 26 de octubre se trabó durante las últimas horas.
Después de que se produjeran los primeros intercambios para llegar a un entendimiento en la ciudad de Buenos Aires, libertarios y macristas no logran acercar posiciones para oficializar el armado de un frente conjunto el próximo jueves, cuando vencerá el plazo legal para anotar las alianzas.
</INPUT>
<CATEGORY>politica</CATEGORY>
</EXAMPLE>

<EXAMPLE>
<INPUT>
El sueño de un Amazon local y una pelea sin tregua contra el cisne negro
El Gobierno confía en que el ecosistema comercial que armó blinde parcialmente los precios contra la suba del tipo de cambio; el FMI
empeoró la expectativa de déficit de cuenta corriente, anticipo de que el dólar seguirá siendo un problema.
</INPUT>
<CATEGORY>economía</CATEGORY>
</EXAMPLE>

Complete the following text with **ONLY** one of these categories: politica, economía. Surround the category with "(" and ")".
<EXAMPLE>
<INPUT>
La suba del dólar tendrá bajo impacto en la inflación de los próximos meses según los analistas que releva el BCRA
</INPUT>

What is the category?
<end_of_turn>
<start_of_turn>model
"""
# Generar texto en base al prompt
response = gemma_lm.generate(prompt_text, strip_prompt=True, stop_token_ids=stop_token)
print(response)

(economía)
